# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 필요한 라이브러리를 설치합니다.
* 로컬 data 폴더에서 데이터셋을 로드합니다.

In [1]:
# 데이터셋 경로 설정
# 로컬 data 폴더에서 데이터를 로드합니다.
import os

# 프로젝트 루트 디렉토리 기준으로 data 폴더 경로 설정
project_root = os.path.abspath('..')  # shared 폴더에서 상위 폴더로 이동
data_path = os.path.join(project_root, 'data')

print(f"데이터 경로: {data_path}")
print(f"데이터 폴더 존재 여부: {os.path.exists(data_path)}")

데이터 경로: /home/james/doc-classification/computervisioncompetition-cv3/data
데이터 폴더 존재 여부: True


In [2]:
# 데이터 폴더 내용 확인
# data 폴더에 있는 파일들을 확인합니다.
import os

if os.path.exists(data_path):
    print("데이터 폴더 내용:")
    for item in os.listdir(data_path):
        item_path = os.path.join(data_path, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")
else:
    print("⚠️ 데이터 폴더가 존재하지 않습니다. 경로를 확인해주세요.")

데이터 폴더 내용:
  📁 test/
  📄 meta.csv
  📄 sample_submission.csv
  📁 train/
  📄 train.csv


In [3]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [4]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [5]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [6]:
# 이미지 데이터셋 클래스 정의
# PyTorch의 Dataset 클래스를 상속받아 커스텀 데이터셋 구현
class ImageDataset(Dataset):
    """
    문서 이미지 분류를 위한 커스텀 데이터셋 클래스
    
    Args:
        csv: CSV 파일 경로 (이미지 파일명과 라벨 정보 포함)
        path: 이미지 파일들이 저장된 디렉토리 경로
        transform: 이미지 전처리를 위한 변환 함수 (선택사항)
    """
    
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values  # CSV 파일을 읽어 numpy 배열로 변환
        self.path = path                   # 이미지 파일 경로
        self.transform = transform         # 이미지 변환 함수

    def __len__(self):
        """데이터셋의 전체 샘플 수 반환"""
        return len(self.df)

    def __getitem__(self, idx):
        """
        주어진 인덱스의 데이터 샘플 반환
        
        Args:
            idx: 데이터 인덱스
            
        Returns:
            tuple: (이미지 텐서, 타겟 라벨)
        """
        name, target = self.df[idx]                                    # 파일명과 타겟 추출
        img = np.array(Image.open(os.path.join(self.path, name)))     # 이미지 로드
        
        if self.transform:
            img = self.transform(image=img)['image']                   # 이미지 변환 적용
            
        return img, target

In [7]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [8]:
# device 설정 - GPU가 사용 가능하면 GPU, 아니면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# data config - 데이터 경로 설정
# 이미 위에서 설정한 data_path 변수를 사용
print(f"데이터 경로: {data_path}")

# model config - 사용할 모델 설정
model_name = 'resnet34'  # 다른 옵션: 'resnet50', 'efficientnet-b0', etc.

# training config - 학습 관련 하이퍼파라미터
img_size = 32        # 입력 이미지 크기
LR = 1e-3           # 학습률 (Learning Rate)
EPOCHS = 1          # 학습 에포크 수
BATCH_SIZE = 32     # 배치 크기
num_workers = 0     # 데이터 로더 워커 수

사용 디바이스: cuda
데이터 경로: /home/james/doc-classification/computervisioncompetition-cv3/data


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [9]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [10]:
# Dataset 정의 - 학습 및 테스트 데이터셋 생성
# 로컬 data 폴더의 파일들을 사용하여 데이터셋 생성
trn_dataset = ImageDataset(
    os.path.join(data_path, "train.csv"),          # 학습 데이터 CSV 파일 경로
    os.path.join(data_path, "train/"),             # 학습 이미지 폴더 경로
    transform=trn_transform                         # 학습용 데이터 변환
)

tst_dataset = ImageDataset(
    os.path.join(data_path, "sample_submission.csv"),  # 테스트 데이터 CSV 파일 경로
    os.path.join(data_path, "test/"),                  # 테스트 이미지 폴더 경로
    transform=tst_transform                             # 테스트용 데이터 변환
)

print(f"학습 데이터셋 크기: {len(trn_dataset)}, 테스트 데이터셋 크기: {len(tst_dataset)}")

학습 데이터셋 크기: 1570, 테스트 데이터셋 크기: 3140


In [11]:
# DataLoader 정의 - 배치 단위로 데이터를 로드하는 데이터 로더 생성
# 학습용 데이터 로더: 데이터 순서를 섞어서 로드
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,    # 배치 크기
    shuffle=True,             # 데이터 순서 섞기 (학습 시 중요)
    num_workers=num_workers,  # 멀티프로세싱 워커 수
    pin_memory=True,          # GPU 메모리 최적화
    drop_last=False           # 마지막 배치가 작아도 포함
)

# 테스트용 데이터 로더: 데이터 순서를 유지하여 로드
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,    # 배치 크기
    shuffle=False,            # 데이터 순서 유지 (예측 시 중요)
    num_workers=0,            # 워커 수 (예측 시에는 보통 0으로 설정)
    pin_memory=True           # GPU 메모리 최적화
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [12]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34-43635321.pth" to /home/james/.cache/torch/hub/checkpoints/resnet34-43635321.pth


In [13]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

  0%|          | 0/50 [00:00<?, ?it/s]

: 

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
# 테스트 데이터에 대한 추론 수행
preds_list = []

model.eval()  # 모델을 평가 모드로 설정 (드롭아웃, 배치 정규화 비활성화)

# 테스트 데이터 배치별로 예측 수행
for image, _ in tqdm(tst_loader, desc="추론 진행 중"):
    image = image.to(device)  # 이미지를 GPU/CPU로 이동

    with torch.no_grad():     # 그래디언트 계산 비활성화 (메모리 절약, 속도 향상)
        preds = model(image)  # 모델 예측 수행
    
    # 예측 결과에서 가장 높은 확률의 클래스 선택
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

In [ ]:
# 예측 결과를 DataFrame으로 정리
# 테스트 데이터의 ID와 예측된 타겟으로 결과 DataFrame 생성
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list  # 예측 결과로 타겟 값 업데이트

print(f"예측 완료: {len(pred_df)}개 샘플")

In [ ]:
# 제출 파일 형식 검증 - sample_submission.csv와 예측 결과의 ID가 일치하는지 확인
sample_submission_df = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
assert (sample_submission_df['ID'] == pred_df['ID']).all(), "ID가 일치하지 않습니다!"
print("✅ 제출 파일 형식 검증 완료")

In [ ]:
# 예측 결과를 CSV 파일로 저장
output_file = "pred.csv"
pred_df.to_csv(output_file, index=False)
print(f"✅ 예측 결과가 '{output_file}' 파일로 저장되었습니다.")

In [ ]:
pred_df.head()